In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
# import streamlit as st
import os
from langchain_groq import ChatGroq
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [25]:
import streamlit as st

In [2]:
from PyPDF2 import PdfReader
import os 

In [3]:
os.environ["GROQ_API_KEY"] = "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"

In [4]:
llm = ChatGroq(temperature=1, model_name="llama3-70b-8192")

In [5]:
pwd


'C:\\Users\\user\\Desktop\\monenco'

In [6]:
uploaded_file = './سوالات.pdf'

In [7]:
def get_pdf_text(pdf):
    pdf_reader = PdfReader(pdf)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [8]:
data  = get_pdf_text('./سوالات.pdf')

In [9]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=900,
        chunk_overlap=100,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [10]:
splits = get_text_chunks(data)

In [11]:
splits

['يکي از آيتم هاي حکم حقوقي در شرکت موننکو، اياب و ذهاب مي باشد که براساس ابالغيه گروه مپنا که مبناي آن رده هاي\nسازماني مي باشند اعمال مي گردد. رده ها از کارمند تا مدير ارشد و مبالغ آن محرمانه مي  باشد.  \nگام نخست اين است که پست سازماني مد نظر در چارت سازماني معاونت مربوطه تعريف شده باشد. سپس معاونت مربوطه فرم  \nاعالم نياز جذب را تکميل و براي معاونت منابع انساني ارسال مي نمايد. معاونت منابع انساني پس از کنترل و تکميل فرم آن را  \nبراي مديريت محترم عامل ارسال کرده و پس از تاييد مديريت محترم عامل ثبت آگهي و دريافت رزومه آغاز ميگردد.  \nرزومه هاي منتخب را به معاونت منابع انساني ارسال کرده و اين رز ومه ها در فراگستر ثبت و دعوت به مصاحبه ميشوند و بعد از  \nتاييد مصاحبه فني، جهت تعيين حقوق و توافق حقوق و مزايا به معاونت منابع انساني ارجاع مي شوند. معاونت منابع انساني با   \nافراد مورد نظر تماس گرفته و حقوق و مزايا ( کارانه  -پاداش  –رفاهي) را به ايشان توضيح داده  و در صورت تاييد فرد رزومه به',
 'مرحله کانون ارزيابي و در صورت تاييد کانون ، فرد منتخب نهايي به مديرعامل ارجاع مي گردد.  \nطبق 

In [12]:
def get_vectorstore(text_chunks):
    embeddings = HuggingFaceEmbeddings()
    knowledge_base = FAISS.from_texts(text_chunks,embeddings)
    return knowledge_base

In [13]:
# import sentence-transformers

In [14]:
vector_db = get_vectorstore(splits)

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [15]:
vector_db

In [16]:
QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an Iranian AI language model assistant. Your task is to generate three
            different versions of the given user question to retrieve relevant documents from
            a vector database. By generating multiple perspectives on the user question, your
            goal is to help the user overcome some of the limitations of the distance-based
            similarity search. Provide these alternative questions separated by newlines.
            Original question: {question}""",
    )

In [17]:
retriever = MultiQueryRetriever.from_llm(
        vector_db.as_retriever(),
        llm,
        prompt=QUERY_PROMPT
    )

In [18]:
data

'  يکي از آيتم هاي حکم حقوقي در شرکت موننکو، اياب و ذهاب مي باشد که براساس ابالغيه گروه مپنا که مبناي آن رده هاي\nسازماني مي باشند اعمال مي گردد. رده ها از کارمند تا مدير ارشد و مبالغ آن محرمانه مي  باشد.  \nگام نخست اين است که پست سازماني مد نظر در چارت سازماني معاونت مربوطه تعريف شده باشد. سپس معاونت مربوطه فرم  \nاعالم نياز جذب را تکميل و براي معاونت منابع انساني ارسال مي نمايد. معاونت منابع انساني پس از کنترل و تکميل فرم آن را  \nبراي مديريت محترم عامل ارسال کرده و پس از تاييد مديريت محترم عامل ثبت آگهي و دريافت رزومه آغاز ميگردد.  \nرزومه هاي منتخب را به معاونت منابع انساني ارسال کرده و اين رز ومه ها در فراگستر ثبت و دعوت به مصاحبه ميشوند و بعد از  \nتاييد مصاحبه فني، جهت تعيين حقوق و توافق حقوق و مزايا به معاونت منابع انساني ارجاع مي شوند. معاونت منابع انساني با   \nافراد مورد نظر تماس گرفته و حقوق و مزايا ( کارانه  -پاداش  –رفاهي) را به ايشان توضيح داده  و در صورت تاييد فرد رزومه به  \nمرحله کانون ارزيابي و در صورت تاييد کانون ، فرد منتخب نهايي به مديرعامل ارجاع مي گردد.  \nطبق 

In [19]:
WRITER_SYSTEM_PROMPT = "You are an IRANIAN AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."  # noqa: E501
    # Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
RESEARCH_REPORT_TEMPLATE = """Information:
    --------
    يکي از آيتم هاي حکم حقوقي در شرکت موننکو، اياب و ذهاب مي باشد که براساس ابالغيه گروه مپنا که مبناي آن رده هاي 
سازماني مي باشند اعمال مي گردد. رده ها از کارمند تا مدير ارشد و مبالغ آن محرمانه مي باشد. 
گام نخست اين است که پست سازماني مد نظر در چارت سازماني معاونت مربوطه تعريف شده باشد. سپس معاونت مربوطه فرم 
اعالم نياز جذب را تکميل و براي معاونت منابع انساني ارسال مي نمايد. معاونت منابع انساني پس از کنترل و تکميل فرم آن را 
براي مديريت محترم عامل ارسال کرده و پس از تاييد مديريت محترم عامل ثبت آگهي و دريافت رزومه آغاز ميگردد. 
رزومه هاي منتخب را به معاونت منابع انساني ارسال کرده و اين رز ومه ها در فراگستر ثبت و دعوت به مصاحبه ميشوند و بعد از 
تاييد مصاحبه فني، جهت تعيين حقوق و توافق حقوق و مزايا به معاونت منابع انساني ارجاع مي شوند. معاونت منابع انساني با 
افراد مورد نظر تماس گرفته و حقوق و مزايا ) کارانه -پاداش –رفاهي( را به ايشان توضيح داده و در صورت تاييد فرد رزومه به 
مرحله کانون ارزيابي و در صورت تاييد کانون ، فرد منتخب نهايي به مديرعامل ارجاع مي گردد. 
طبق قانون کار و قوانين داخلي گروه مپنا و موننکو، در صورتي که تاريخ عقد بعد از تاريخ استخدام در موننکو باشد همکار مي 
تواند از 3 روز مرخصي اضطراري – ازدواج استفاده نمايد. الزم به ذکر مي باشد بهره مندي از مرخصي ازدواج نيز مانند ساير 
مرخصي ها مي بايست با تاييد مدير مربوطه انجام گردد. 
روال در متن گواهي سابقه ي کار درج آخرين پست سازماني در حکم مي باشد. چنانچه درخواست اضافه نمودن شرح شغل را 
داشته باشيد بايد حداکثر 3 شرح وظايف داراي اولويت را پس از تاييد مدير و يا سرپرست مستقيم به معاونت منابع انساني 
ارسال نماييد تا در گواهي سابقه ي کار اعمال گردد. 
با استعالمي که معاونت منابع انساني از معاونت مالي اخذ مي نمايد، امکان درج ميانگين ماهيانه ميسر است. 
مطابق مقررات داخلي موننکو در صورت بروز بيماري براي همکار، مي تواند تا سقف 6 روز از مرخصي استعالجي با ارائه ويزيت 
پزشک و اعالم نياز به استراحت در آن و ارسال آن به امور کارکنان استفاده نمايد و حقوق بيماري تا سقف 6 روز توسط موننکو 
پرداخت مي گردد. الزم به ذکر است براي استفاده از مرخصي استعالجي بيش از سقف اعالم شده، مي بايست مدارک مربوط به 
بيماري را در قسمت مربوطه سابت تامين اجتماعي بارگذاري گردد و پس از تاييد کميسيون پزشکي حقوق ايام بيماري را دريافت 
مي نمايند 
بعد از تکميل پرونده استخدامي که مهم ترين آنها گواهي سابقه کار –ليست بيمه -مدرک تحصيلي مي باشد، معاونت منابع 
انساني نسبت به ارزيابي حقوق با توجه به سابقه کار و مدرک اقدام مي نمايد. فرم ارزيابي ابتدا به امضاء مدير امور کارکنان سپس 
معاونت مربوطه و نماينده مدير عامل و در نهايت با تاييد مديريت محترم عامل مي رسد و قرارداد همکار بر اساس آن صادر مي 
گردد. 
مطابق ابالغيه گروه مپنا که در ابتداي هر سال انجام مي گيرد به مادراني که فرزند زير 5 سال ) يعني تا قبل از پيش د بستاني ( 
دارند هزينه اي به عنوان هزينه مهد کودک تعلق ميگيرد که در فيش حقوق اعمال ميگردد و تاريخ پايان آن 5 سالگي اوالد مي 
باشد. 
معاونين در موننکو مي توانند در صورت اعتراض و يا بازنگري حقوق پرسنل خود به معاون منابع انساني درخواستي را ارسال 
نمايند. معاونت منابع انساني با بررسي افراد مشابه در شرکت از لحاظ سابقه و تحصيالت با فرد مورد نظر، جدول مقايسه اي را 
تنظيم مي نمايد. در جدول مربوطه حداقل، ميانگين و حداکثر حقوق مشابهين درج ميگردد و چنانچه فرد مورد نظر از حداقل 
جدول مقايسه اي کمتر باشد در خصوص فرد مورد نظر تصميم گيري مي شود. نهايتا تصميمي که در مورد افزايش/عدم افزايش 
فرد گرفته ميشود به معاون درخواست کننده اطالع رساني مي گردد. 
بر اساس دستورالعمل کارانه و امتياز اختصاص يافته به هر معاونت، بودجه اي بر اساس حقوق کارکنان در هر معاونت نظر گرفته 
ميشود. شايستگي هاي نيز تدوين شده است که براي هر فرد بر اساس شايستگي ها امتياز در نظر گرفته ميشود. 
بر اساس سياست هر معاونت ميتواند 1.2 تا 1.5 برابر بودجه هر نفر به شرط رعايت سقف بودجه معاونت تعيين گردد. 
مطابق دستورالعمل جا به جايي و فرم نقل و انتقال درون سازماني، ابتدا معاونت مقصد درخواست انتقال را براي معاونت منابع 
انساني ارسال کرده در صورت تاييد معاونت منابع انساني فرم را به معاونت مبدا و در صورت تاييد معاونت مبدا، فرم براي همکار 
ارسال مي گردد تا رضايت خود را اعالم نمايد و سپس براي مرحله جابه جايي نهايي به معاونت منابع انساني ارسال مي گردد. در 
صورت عدم تاييد در معاونت مبدا موضوع منتفي مي باشد. 
به محض دريافت نامه خاتمه همکاري در امور کارکنان، فرآيند تسويه حساب همکار در فراگستر به صورت الکترونيکي آغاز مي 
گردد در صورتي که تمام مراحل اعم از مديرگروه/ معاونت مربوطه - کتابخانه/آموزش/ فناوري اطالعات / وام/مرخصي و ... انجام 
شود به معاونت منابع انساني ارسال شده و در صورت تاييد به امور مالي جهت محاسبات و اولويت پرداخت ارسال مي گردد. 
مطابق قانون کار براي همکاراني که صاحب فرزند ميشوند )محدوديت در تعداد فرزند نيست( و بيش از 720 روز سابقه بيمه 
تامين اجتماعي داشته باشند ، آيتم حق اوالد که معادل 3 روز حداقل حقوق وزارت کار )يا %10 حداقل حقوق( در حکم حقوقي 
ايشان اضافه مي گردد. هم چنين براي همکاراني که فرزند باالي 18 سال داشته باشند و فرزند اشتغال به تحصيل نداشته باشد و 
يا همکار بازنشسته گردد آيتم حق اوالد از حکم ايشان حذف مي گردد. 
در صورتي که همکار آقا باشد مي بايست مدارک شناسنامه اي و کارت ملي همسر را عالوه بر امور کارکنان به بيمه نيز ارسال 
نمايد. در صورتي که همکار خانم باشد و درخواست بيمه براي همسر را داشته باشد مدارک را به بيمه نيز ارسال مي نمايد که اين 
مورد براي همکار خانم داراي هزينه مي باشد. هم چنين فايل آشنايي دريافت هديه ازدواج تامين اجتماعي در صورت داشتن 
720 روز سابقه بيمه براي همکار ارسال مي گردد
       --------
    Using the above information, answer the following question or topic: "{question}" in a short manner-- \
    The answer should focus on the answer to the question, should be well structured, informative, \
    in depth, with facts and numbers if available and a minimum of 150 words and a maximum of 300 words.
    You should strive to write the report using all relevant and necessary information provided.
    You must write the report with markdown syntax.
    You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
    You must write the sources used in the context. if any article is used, mentioned in the end.
    Please do your best, this is very important to my career.
    You have to talk in persian. Always assume you have to spean farsi until the user ask you to speak other language."""   # noqa: E501


In [20]:
prompt = ChatPromptTemplate.from_messages(
        [
            ("system", WRITER_SYSTEM_PROMPT),
            ("user", RESEARCH_REPORT_TEMPLATE),
        ]
    )

In [21]:
chain = (
            {"text": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
    )


In [26]:
with st.chat_message("user"):
    st.write("Hello 👋")

2024-07-07 13:42:25.731 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [22]:
input = ' چگونه است؟'

In [23]:
answer = chain.invoke(
        {
            "question": input
        }
    )

In [24]:
answer

'**چگونه است؟**\n================\n\nبر اساس اطلاعات ارائه شده، فرایند استخدام در شرکت موننکو از چند مرحله تشکیل شده است. ابتدا پست سازمانی در چارت سازمانی معاونت مربوطه تعریف شده و سپس معاونت مربوطه فرم اعلام نیازمندی جذب را تکمیل و برای معاونت منابع انسانی ارسال می کند. پس از控制 و تکمیل فرم، آن را برای مدیریت محترم عامل ارسال کرده و پس از تأیید، ثبت آگهی و دریافت رزومه آغاز می گردد.\n\nبعد از انتخاب رزومه ها، مصاحبه فنی برگزار شده و در صورت تأیید، حقوق و مزایا به معاونت منابع انسانی ارجاع می شود. معاونت منابع انسانی با افراد مورد نظر تماس گرفته و حقوق و مزایا را به آنان توضیح داده و در صورت تأیید، فرد منتخب نهایی به مدیر عامل ارجاع می گردد.\n\nهمچنین، طبق قانون کار و قوانین داخلی گروه مپنا و موننکو، در صورتی که تاریخ عقد بعد از تاریخ استخدام در موننکو باشد، همکار می تواند از ۳ روز مرخصی اضطراری – ازدواج استفاده نماید. در این فرایند، شرکت موننکو به کارکنان خود حقوق و مزایایی را ارائه می دهد که شامل حق اوالد، پاداش، رفاهی و سایر مزایا می باشد.\n\nبنابراین، فرایند استخدام در شرکت موننکو 